In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [8]:
final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"].sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [9]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [10]:
final_df.groupby("user_id")["total_amount"].sum().gt(1000).sum()


2544

In [32]:
final_df["rating_bucket"] = pd.cut(
    final_df["rating"], [3,3.5,4,4.5,5]
)

final_df.groupby("rating_bucket")["total_amount"].sum()



C:\Users\RAHUL\AppData\Local\Temp\ipykernel_34100\282108284.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_bucket")["total_amount"].sum()


rating_bucket
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
(4.0, 4.5]    1960326.26
(4.5, 5.0]    2197030.75
Name: total_amount, dtype: float64

In [12]:
final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [13]:
restaurants.groupby("cuisine")["restaurant_id"].nunique()


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [14]:
final_df.groupby("cuisine")["total_amount"].sum()


cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

In [15]:
round(
    final_df[final_df["membership"]=="Gold"].shape[0]
    / final_df.shape[0] * 100
)


50

In [17]:
final_df.column


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_bucket'],
      dtype='object')

In [33]:
import pandas as pd
import sqlite3
import json

orders = pd.read_csv('orders.csv')

rest_stats = orders.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    order_count=('order_id', 'count')
)

less_than_20 = rest_stats[rest_stats['order_count'] < 20]
options = [
    'Grand Cafe Punjabi', 
    'Grand Restaurant South Indian', 
    'Ruchi Mess Multicuisine', 
    'Ruchi Foods Chinese'
]

result = less_than_20.loc[less_than_20.index.intersection(options)]

print("Analysis of the specific options:")
print(result)



Analysis of the specific options:
                     avg_order_value  order_count
restaurant_name                                  
Ruchi Foods Chinese       686.603158           19


In [34]:
combo_rev = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
options = [
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Gold', 'Indian'),
    ('Regular', 'Chinese')
]

results = {str(opt): combo_rev.get(opt, 0) for opt in options}
sorted_results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))

for combo, revenue in sorted_results.items():
    print(f"{combo}: ₹{revenue:,.2f}")


('Gold', 'Italian'): ₹1,005,779.05
('Regular', 'Indian'): ₹992,100.27
('Gold', 'Indian'): ₹979,312.31
('Regular', 'Chinese'): ₹952,790.91


In [23]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum()


C:\Users\RAHUL\AppData\Local\Temp\ipykernel_34100\1682251932.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [24]:
final_df[final_df["membership"]=="Gold"].shape[0]

4987

In [25]:
round(final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum())

1889367

In [26]:
final_df["user_id"].nunique()

2883

In [27]:
round(final_df[final_df["membership"]=="Gold"]["total_amount"].mean(),2)

797.15

In [28]:
final_df[final_df["rating"]>=4.5].shape[0]

3374

In [29]:
top_city = final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"].sum().idxmax()

final_df[(final_df["membership"]=="Gold") & (final_df["city"]==top_city)].shape[0]

1337

In [35]:
import pandas as pd
import sqlite3
import json

orders = pd.read_csv('orders.csv')

with open('users.json', 'r') as f:
    users_data = json.load(f)
users = pd.DataFrame(users_data)

conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    conn.executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

merged_df = pd.merge(orders, users, on='user_id', how='left')

final_df = pd.merge(
    merged_df, 
    restaurants, 
    on='restaurant_id', 
    how='left', 
    suffixes=('_order_file', '_master_file')
)

final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("File 'final_food_delivery_dataset.csv' has been generated successfully!")
print(f"Total Rows: {len(final_df)}")
print("Columns included:", final_df.columns.tolist())

File 'final_food_delivery_dataset.csv' has been generated successfully!
Total Rows: 10000
Columns included: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_order_file', 'name', 'city', 'membership', 'restaurant_name_master_file', 'cuisine', 'rating']


In [36]:
import os
if os.path.exists('final_food_delivery_dataset.csv'):
    print("The file is ready for download in your current directory!")
else:
    print("File not found. Please check your save path.")

The file is ready for download in your current directory!


In [37]:
from IPython.display import FileLink
FileLink('final_food_delivery_dataset.csv')

C:\Users\RAHUL\final_food_delivery_dataset.csv